In [25]:
# import_sqlite_to_postgres.py (HOÀN HẢO - ESCAPE %s)
import sqlite3
import pandas as pd
import psycopg2
from psycopg2.extras import execute_values
import os

In [26]:
# ===================== CẤU HÌNH =====================
SQLITE_DB = r'D:\Laptrinh\Python\.WSE\Warehouse_app\CODE COVER DATA\database\Inventory_management_EOL.db'
TABLE_NAME = "Key_All"
TEAM_ID = 1

POSTGRES_DSN = "host=172.23.8.153 dbname=warehouse_db user=postgres password=wsepc port=5432 sslmode=disable"

COLUMNS_TO_MELT = [
    "model", "process", "groups", "material",
    "unit", "storage_location", "status"
]

In [27]:
# lấy list bảng của POSTGRES_DSN
def get_postgres_tables(dsn):
    with psycopg2.connect(dsn) as conn:
        with conn.cursor() as cur:
            cur.execute("""
                SELECT table_name
                FROM information_schema.tables
                WHERE table_schema = 'public'
            """)
            tables = [row[0] for row in cur.fetchall()]
    return tables
# =====================================================
print(get_postgres_tables(POSTGRES_DSN))

['options', 'teams', 'users', 'inventory_entries', 'audit_log']


In [28]:
# Lấy cột của bảng table_postgresql_name trong PostgreSQL
table_postgresql_name = 'options'

def get_postgres_table_columns(dsn, table_name):
    with psycopg2.connect(dsn) as conn:
        with conn.cursor() as cur:
            cur.execute(f"""
                SELECT column_name
                FROM information_schema.columns
                WHERE table_name = %s
            """, (table_name,))
            columns = [row[0] for row in cur.fetchall()]
    return columns
print(get_postgres_table_columns(POSTGRES_DSN, table_postgresql_name))

['id', 'team_id', 'category', 'value', 'is_active', 'sort_order', 'created_at', 'created_by', 'updated_at', 'updated_by']


In [29]:
# đọc dữ liệu table_postgresql_name từ PostgreSQL
def read_postgres_table(dsn, table_name):
    with psycopg2.connect(dsn) as conn:
        with conn.cursor() as cur:
            cur.execute(f"SELECT * FROM {table_name}")
            rows = cur.fetchall()
    return rows

# print(read_postgres_table(POSTGRES_DSN, table_postgresql_name))
# chuyển dữ liệu table_postgresql_name thành DataFrame
def postgres_table_to_dataframe(dsn, table_name):
    with psycopg2.connect(dsn) as conn:
        query = f"SELECT * FROM {table_name}"
        df = pd.read_sql_query(query, conn)
    return df
df = postgres_table_to_dataframe(POSTGRES_DSN, table_postgresql_name)
df

C:\Users\sscn203\AppData\Local\Temp\ipykernel_13252\554903517.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,id,team_id,category,value,is_active,sort_order,created_at,created_by,updated_at,updated_by


In [30]:
def read_sqlite_table():
    print(f"Đang đọc {TABLE_NAME} từ SQLite...")
    if not os.path.exists(SQLITE_DB):
        raise FileNotFoundError(SQLITE_DB)
    
    conn = sqlite3.connect(SQLITE_DB)
    query = f"SELECT {', '.join(COLUMNS_TO_MELT)} FROM {TABLE_NAME}"
    df = pd.read_sql_query(query, conn)
    conn.close()
    print(f"Đọc: {len(df)} dòng.")
    return df

In [31]:
df = read_sqlite_table()
df

Đang đọc Key_All từ SQLite...
Đọc: 181 dòng.


,model,process,groups,material,unit,storage_location,status
0,AZ,AF-Tanshi,Setup Parts,Metal,pcs,Box 001,Available
1,BRW,ACF,Standard Jig,Plastic,set,Box 002,Unavailable
2,BRX,FPC Reinf.UV,Screws,Other,box,Box 003,None
3,GA,Stiffener Bottom,Air Tubes,None,cm,Box 004,None
4,BE,Stiffener Castellation,Electric Wires,None,Other,Box 005,None
...,...,...,...,...,...,...,...
176,None,None,None,None,None,Vendor VE16,None
177,None,None,None,None,None,Vendor VE17,None
178,None,None,None,None,None,Vendor VE18,None
179,None,None,None,None,None,Vendor VE19,None


In [32]:
def prepare_melted_data(df):
    print("Chuyển cột → hàng bằng pd.melt...")
    df_melted = pd.melt(df, var_name='category', value_name='value')
    
    df_melted = df_melted[
        df_melted['value'].notnull() &
        (df_melted['value'].astype(str).str.strip() != '') &
        (df_melted['value'].astype(str).str.lower() != 'none')
    ].copy()

    df_melted['team_id'] = TEAM_ID
    df_melted['is_active'] = True
    df_melted['sort_order'] = df_melted.groupby('category').cumcount() + 1

    # LOẠI TRÙNG
    before = len(df_melted)
    df_melted = df_melted.drop_duplicates(subset=['team_id', 'category', 'value'])
    df_melted['created_at'] = pd.Timestamp.now()
    df_melted['created_by'] = 1
    df_melted['updated_at'] = pd.Timestamp.now()
    df_melted['updated_by'] = None
    print(f"Loại trùng: {before} → {len(df_melted)} dòng.")

    return df_melted[['team_id', 'category', 'value', 'sort_order', 'is_active', 'created_at', 'created_by', 'updated_at', 'updated_by']]

In [33]:
print("BẮT ĐẦU IMPORT (ESCAPE %s)\n")
df_melted = prepare_melted_data(df)
df_melted

BẮT ĐẦU IMPORT (ESCAPE %s)

Chuyển cột → hàng bằng pd.melt...
Loại trùng: 242 → 242 dòng.


,team_id,category,value,sort_order,is_active,created_at,created_by,updated_at,updated_by
0,1,model,AZ,1,True,2025-11-06 11:03:22.537677,1,2025-11-06 11:03:22.541120,None
1,1,model,BRW,2,True,2025-11-06 11:03:22.537677,1,2025-11-06 11:03:22.541120,None
2,1,model,BRX,3,True,2025-11-06 11:03:22.537677,1,2025-11-06 11:03:22.541120,None
3,1,model,GA,4,True,2025-11-06 11:03:22.537677,1,2025-11-06 11:03:22.541120,None
4,1,model,BE,5,True,2025-11-06 11:03:22.537677,1,2025-11-06 11:03:22.541120,None
...,...,...,...,...,...,...,...,...,...
1083,1,storage_location,Vendor VE18,179,True,2025-11-06 11:03:22.537677,1,2025-11-06 11:03:22.541120,None
1084,1,storage_location,Vendor VE19,180,True,2025-11-06 11:03:22.537677,1,2025-11-06 11:03:22.541120,None
1085,1,storage_location,Vendor VE20,181,True,2025-11-06 11:03:22.537677,1,2025-11-06 11:03:22.541120,None
1086,1,status,Available,1,True,2025-11-06 11:03:22.537677,1,2025-11-06 11:03:22.541120,None


In [34]:
def insert_to_postgres(df_melted):
    print("Kết nối PostgreSQL...")
    conn = psycopg2.connect(POSTGRES_DSN)
    cur = conn.cursor()

    # 1. XÓA DỮ LIỆU CŨ
    print(f"Xóa team_id={TEAM_ID}...")
    cur.execute("DELETE FROM options WHERE team_id = %s", (TEAM_ID,))

    # 2. CHÈN DỮ LIỆU MỚI - DÙNG executemany
    records = [
        (
            row['team_id'],
            row['category'],
            str(row['value']).strip(),
            row['sort_order'],
            row['is_active'],
            row['created_at'],
            row['created_by'],
            row['updated_at'],
            row['updated_by']
        )
        for _, row in df_melted.iterrows()
    ]
    
    insert_sql = """
        INSERT INTO options (team_id, category, value, sort_order, is_active, created_at, created_by, updated_at, updated_by)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (team_id, category, value) DO NOTHING
    """
    
    print(f"Chèn {len(records)} bản ghi bằng executemany...")
    cur.executemany(insert_sql, records)

    conn.commit()
    cur.close()
    conn.close()
    print("HOÀN TẤT!")

In [35]:
df_melted.head(10)

,team_id,category,value,sort_order,is_active,created_at,created_by,updated_at,updated_by
0,1,model,AZ,1,True,2025-11-06 11:03:22.537677,1,2025-11-06 11:03:22.541120,None
1,1,model,BRW,2,True,2025-11-06 11:03:22.537677,1,2025-11-06 11:03:22.541120,None
2,1,model,BRX,3,True,2025-11-06 11:03:22.537677,1,2025-11-06 11:03:22.541120,None
3,1,model,GA,4,True,2025-11-06 11:03:22.537677,1,2025-11-06 11:03:22.541120,None
4,1,model,BE,5,True,2025-11-06 11:03:22.537677,1,2025-11-06 11:03:22.541120,None
5,1,model,BEF,6,True,2025-11-06 11:03:22.537677,1,2025-11-06 11:03:22.541120,None
6,1,model,BEL,7,True,2025-11-06 11:03:22.537677,1,2025-11-06 11:03:22.541120,None
7,1,model,BEP,8,True,2025-11-06 11:03:22.537677,1,2025-11-06 11:03:22.541120,None
8,1,model,BEY,9,True,2025-11-06 11:03:22.537677,1,2025-11-06 11:03:22.541120,None
9,1,model,BEZ,10,True,2025-11-06 11:03:22.537677,1,2025-11-06 11:03:22.541120,None


In [36]:
# Lấy dataframe chuẩn bị chèn vào bảng options postgreSQL
insert_to_postgres(df_melted)
# print("\nXONG!")

Kết nối PostgreSQL...
Xóa team_id=1...
Chèn 242 bản ghi bằng executemany...
HOÀN TẤT!
